In [1]:
!rm -rf shared

In [2]:
!uv pip install -e ../python
!uv pip install pandas

Using Python 3.12.7 environment at: /Users/madhavajay/dev/biovault-beaver/main/.venv
Resolved 10 packages in 1.41s                                        
Prepared 1 package in 92ms                                               
Uninstalled 1 package in 0.63ms
Installed 1 package in 1ms1.23 (from file:///Users/madhavaja
 ~ biovault-beaver==0.1.23 (from file:///Users/madhavajay/dev/biovault-beaver/main/python)
Using Python 3.12.7 environment at: /Users/madhavajay/dev/biovault-beaver/main/.venv
Audited 1 package in 3ms


In [3]:
import beaver
import pandas as pd

In [4]:
bv = beaver.connect(folder="shared", user="alice")

In [5]:
@bv
class A:
    b: int

In [6]:
res = A.send(user="bob")
print(res.path)

shared/bob/9ed69b9eeca54d438c8e12b3aee02306.beaver


In [7]:
@bv(requirements=["pandas>=2.0.0"])
def sum_df(filename: str) -> float:
    import pandas as pd
    df = pd.read_csv(filename)
    res = df['total'].sum()
    print(f"\nTotal sum: ${res:,.2f}")
    return res

In [8]:
sum_df("mock.csv")


Total sum: $659.00


np.float64(659.0)

In [9]:
res = sum_df.send(user="bob")
print(res.path)

shared/bob/29cb37bb3cf041b182f0ecb64983473c.beaver


In [ ]:
bv.inbox()[0]

In [ ]:
assert False

In [ ]:
x = bv.inbox()[0].load()

In [ ]:
x

In [ ]:
type(x)

In [ ]:
x = bv.inbox()[0]

In [ ]:
type(x)

In [ ]:
x.payload

In [ ]:
y = x.load()

In [ ]:
result = bv.inbox_view()[0]

In [ ]:
result

In [ ]:
r = result.load()

In [ ]:
r

In [ ]:
env, obj = bv.load_by_id(
  envelope_id="835dd8ff4ca54fc7999128a57c4b71e4",
  inject_globals=True,
  globals_ns=globals(),
)

In [ ]:
obj

In [ ]:
env

In [ ]:
@bv
def greet(name): return f"hi {name}"

In [ ]:
res = greet.send(out_dir="shared/bob")
print(res.envelope_id)

In [ ]:
req_path

In [ ]:
# req_id = req_path.stem.split("-", 2)[-1]
# print("sent", req_id)

In [ ]:
assert False

In [ ]:
import beaver, re
from collections import Counter
from pathlib import Path

In [ ]:
bv = beaver.connect(folder="shared", user="ds")  # shared inbox/

In [ ]:
TEXT_FILE = "input.txt"

In [ ]:
def clean_tokens(text: str):
  return re.findall(r"[a-zA-Z]+", text.lower())

In [ ]:
@bv  # adds .send()
def top_bigrams(path: str, k: int = 5):
  text = Path(path).read_text()
  toks = clean_tokens(text)
  bigrams = Counter(zip(toks, toks[1:]))
  return bigrams.most_common(k)

In [ ]:
# Cell 2: run locally to preview
preview = top_bigrams(TEXT_FILE, k=5)
preview

In [ ]:
# Cell 3: send the work/result as a beaver file
res = top_bigrams.send(TEXT_FILE, k=5)

In [ ]:
print("sent:", res.path, "id:", res.envelope_id)

In [ ]:
# On the receiving side, listen_once(autorun=True) will compute and
# write back a reply; or they can load the function without args and
# call it manually.

In [ ]:
@beaver.export(sender="alice", out_dir="outbox", name="greet")
def greet(name): return f"hi {name}"

path = greet.send("Bob")
env_id = path.stem.split("-", 2)[-1]  # or pack(..., envelope_id=...) to
set one
print("sent file", path, "id", env_id)

In [ ]:
# sender
path = beaver.write_envelope(
    beaver.pack(greet, sender="alice", name="greet_func", inputs=["name"], outputs=["greeting"], requirements=["numpy"]), out_dir="outbox",
)
print(path)